### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [2]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]# number of rows in the matrix
    n_movies =ratings_mat.shape[1] # number of movies in the matrix
    num_ratings =n_users*n_movies # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat =np.random.rand(n_users,latent_features) # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features,n_movies)# movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                # if the rating exists
                if(ratings_mat[i,j]>0):
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff=ratings_mat[i,j]-np.dot(user_mat[i,:],movie_mat[:,j])

                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum+=diff**2
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i,k]+=learning_rate*(2*diff*movie_mat[k,j])
                        movie_mat[k,j]+=learning_rate*(2*diff*user_mat[i,k])

        # print results for iteration
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [3]:
user_mat, movie_mat =FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0005, iters=10) # use your function with 4 latent features, lr of 0.005 and 10 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 56.252860
2 		 53.182313
3 		 50.009202
4 		 46.740027
5 		 43.390981
6 		 39.987451
7 		 36.562956
8 		 33.157438
9 		 29.814934
10 		 26.580778


In [4]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 4.30932238  4.11910232  4.23940257  4.9997073 ]
 [ 3.29201158  3.34487571  3.53855169  3.81106486]
 [ 3.53758488  3.0020966   3.3994445   3.76235231]
 [ 3.64608124  3.56482755  3.55775885  4.27767085]
 [ 4.06760389  3.7456097   3.92269319  4.68508826]
 [ 2.14697331  2.33876546  2.16569056  2.7630092 ]
 [ 3.91885518  3.60660454  3.78177624  4.49938769]
 [ 3.7925847   3.73320411  3.83312414  4.47311282]
 [ 3.66604161  3.81115504  3.72799747  4.45065598]
 [ 3.44196985  2.92825499  3.29639212  3.66668517]
 [ 3.55200688  2.90957733  3.11817747  3.90972043]
 [ 4.36065224  3.80800472  4.08201861  4.86637225]
 [ 4.14934198  3.50909642  3.83104548  4.53045415]
 [ 2.78693804  3.10635682  3.11850813  3.37024197]
 [ 3.76133794  3.68478202  3.64739726  4.52485511]
 [ 3.46180159  3.11009046  3.26198608  3.90909388]
 [ 4.30802578  3.94051901  4.33561207  4.77127014]
 [ 3.79096848  4.0352641   3.94137368  4.65088457]
 [ 2.73634468  2.97782669  3.05701845  3.23484948]
 [ 3.8069335   3.85504718  4.01

**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [5]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0005, iters=250)#use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 55.056388
2 		 52.428605
3 		 49.652118
4 		 46.737199
5 		 43.701866
6 		 40.571868
7 		 37.380151
8 		 34.165712
9 		 30.971835
10 		 27.843804
11 		 24.826260
12 		 21.960491
13 		 19.281939
14 		 16.818235
15 		 14.587962
16 		 12.600272
17 		 10.855312
18 		 9.345341
19 		 8.056304
20 		 6.969636
21 		 6.064050
22 		 5.317150
23 		 4.706744
24 		 4.211820
25 		 3.813192
26 		 3.493847
27 		 3.239065
28 		 3.036369
29 		 2.875363
30 		 2.747512
31 		 2.645894
32 		 2.564950
33 		 2.500255
34 		 2.448309
35 		 2.406361
36 		 2.372256
37 		 2.344310
38 		 2.321210
39 		 2.301935
40 		 2.285688
41 		 2.271848
42 		 2.259931
43 		 2.249555
44 		 2.240421
45 		 2.232295
46 		 2.224988
47 		 2.218352
48 		 2.212268
49 		 2.206638
50 		 2.201385
51 		 2.196445
52 		 2.191765
53 		 2.187303
54 		 2.183022
55 		 2.178893
56 		 2.174889
57 		 2.170991
58 		 2.167179
59 		 2.163438
60 		 2.159756
61 		 2.156120
62 		 2.152521
63 		

In [6]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.23677649   8.72274815  10.62573502  10.19311192]
 [  8.76612487   7.29121761   8.70200953   8.73276981]
 [  7.4171578    7.13637075  10.03177289   7.09232107]
 [  9.64686592   8.07971585   9.71345385   9.61534781]
 [  9.05081466   6.98514134   8.79618921   8.45982657]
 [  6.34271054   5.50429342   8.42645097   6.06142335]
 [  8.72530618   8.09002242  10.49221503   8.67540181]
 [  8.763893     7.08753164   7.86911987   8.62637221]
 [  7.83552054   7.25952978   9.59829016   8.24277741]
 [  7.62140772   6.55259355   8.45493848   7.59962706]
 [  8.94482823   7.40863941  10.04925535   8.49960338]
 [  9.4668814    8.46265645  10.49320147   9.32425933]
 [  9.49825017   8.02925207  10.21624588   9.07334636]
 [  6.76023738   5.85740266   6.21194184   6.83250145]
 [  9.92896354   8.48963314   9.60755172  10.07288703]
 [  9.83435537   8.67307568   9.81769553   9.66665609]
 [  8.2922125    7.48629225   8.75884878   8.31815002]
 [  9.66104618   6.40734618   3.062778     9.49844786]
 [  7.3673

**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [7]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,  10.,  10.],
        [ 10.,   4.,   9.,  10.],
        [  8.,   9.,  10.,   5.],
        [  9.,   8.,  10.,  10.],
        [ 10.,   5.,   9.,   9.],
        [  6.,   4.,  10.,   6.],
        [  9.,   8.,  10.,   9.],
        [ 10.,   5.,   9.,   8.],
        [  7.,   8.,  10.,   8.],
        [  9.,   5.,   9.,   7.],
        [  9.,   8.,  10.,   8.],
        [  9.,  10.,  10.,   9.],
        [ 10.,   9.,  10.,   8.],
        [  5.,   8.,   5.,   8.],
        [ 10.,   8.,  10.,  10.],
        [  9.,   9.,  10.,  10.],
        [  9.,   8.,   8.,   8.],
        [ 10.,   8.,   1.,  10.],
        [  5.,   6.,  10.,  10.],
        [  8.,   7.,  10.,   7.]])

In [8]:
# run SVD on the matrix with the missing value
user_mat, movie_mat =FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0005, iters=250) #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 58.153135
2 		 55.981156
3 		 53.656355
4 		 51.177290
5 		 48.548425
6 		 45.780769
7 		 42.892267
8 		 39.907825
9 		 36.858854
10 		 33.782260
11 		 30.718893
12 		 27.711490
13 		 24.802319
14 		 22.030716
15 		 19.430808
16 		 17.029675
17 		 14.846148
18 		 12.890362
19 		 11.164040
20 		 9.661419
21 		 8.370620
22 		 7.275255
23 		 6.356070
24 		 5.592446
25 		 4.963660
26 		 4.449848
27 		 4.032669
28 		 3.695685
29 		 3.424537
30 		 3.206937
31 		 3.032553
32 		 2.892829
33 		 2.780765
34 		 2.690694
35 		 2.618068
36 		 2.559261
37 		 2.511400
38 		 2.472216
39 		 2.439924
40 		 2.413121
41 		 2.390706
42 		 2.371814
43 		 2.355766
44 		 2.342025
45 		 2.330168
46 		 2.319859
47 		 2.310830
48 		 2.302867
49 		 2.295796
50 		 2.289478
51 		 2.283799
52 		 2.278663
53 		 2.273995
54 		 2.269729
55 		 2.265810
56 		 2.262194
57 		 2.258841
58 		 2.255718
59 		 2.252797
60 		 2.250052
61 		 2.247464
62 		 2.245013
63 

In [9]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 10.095444657905777:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [10]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.0005, iters=20)#fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 0.014530
2 		 0.012200
3 		 0.010224
4 		 0.008658
5 		 0.007476
6 		 0.006593
7 		 0.005920
8 		 0.005393
9 		 0.004967
10 		 0.004616
11 		 0.004319
12 		 0.004065
13 		 0.003844
14 		 0.003649
15 		 0.003477
16 		 0.003322
17 		 0.003182
18 		 0.003055
19 		 0.002939
20 		 0.002832


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [20]:
# Replace each of the comments below with the correct values
num_ratings =10852 # How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = 31234148# How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [21]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?


In [19]:
n_users_1000=first_1000_users.shape[0]
n_movies_1000=first_1000_users.shape[1]
#ratings=n_users_1000*n_movies_1000
np.isnan(first_1000_users).sum()

31234148

In [ ]:
missing_ratings=ratings-ratings.isnull().sum()